In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, classification_report


In [4]:
df = pd.read_csv('data\Damage data.csv')
df.drop('Date', axis=1, inplace=True)
df = df.fillna(value={'Pillars':'None','Damage':'None'})
# df['Roof'] = df['Roof'].fillna('Asbestos')
# df['Pillars'] = df['Pillars'].fillna('Concrete')
# df['Walls'] = df['Walls'].fillna('Block')
# df['Damage'] = df['Damage'].fillna('None')
# df['Flood Height'] = df['Flood Height'].fillna(0)



In [7]:
repair_bins = [-float('inf'), 20000, 40000, 60000, 80000, 100000, float('inf')]
value_bins = [0, 2000000, 4000000, 6000000, 8000000, 10000000, 12000000, 14000000, 16000000, 18000000, 20000000, float('inf')]
height_bins = [0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210, 225, 240, 255, 270, 285, 300, 315, 330, 345, 360, 375, 390, 405, 420, 435,float('inf')]

df['Estimated Repair value bins'] = pd.cut(df['Estimated Repair value'], bins=repair_bins, labels=False)
df['Estimated value bins'] = pd.cut(df['Estimated value'], bins=value_bins, labels=False)
df['Flood Height bins'] = pd.cut(df['Flood Height'], bins=height_bins, labels=False)

In [23]:
# num_bins = 5

# df['Estimated Repair value bins'] = pd.cut(df['Estimated Repair value'], bins=num_bins, labels=False)
# df['Estimated value bins'] = pd.cut(df['Estimated value'], bins=num_bins, labels=False)

In [17]:
df.value_counts('Estimated Repair value bins')

Estimated Repair value bins
0    64
1    17
2     9
3     7
4     2
5     2
Name: count, dtype: int64

In [8]:
X_labels = ['Building Age', 'Estimated value bins', 'Floors', 'Building Height', 'Walls', 'Roof', 'Pillars', 'Flood Height']
y_label = ['Estimated Repair value bins']
categorical_cols = ['Walls', 'Roof', 'Pillars']

X = df[X_labels]
y = df[y_label]

In [9]:
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(df[categorical_cols])

encoded_cols = encoder.get_feature_names_out(categorical_cols)
X_encoded_df = pd.DataFrame(X_encoded.toarray(), columns=encoded_cols)

X_numerical = X.drop(columns=categorical_cols)
X_final = pd.concat([X_encoded_df, X_numerical], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)


In [10]:
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100,random_state=42),
    'Linear Regression': LinearRegression(),
    'Decision Tree Regression': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting Regression': GradientBoostingRegressor(random_state=42),
    'K-Nearest Neighbors Regression': KNeighborsRegressor()
}

In [11]:
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"{model_name} MSE: {mse}")

e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest MSE: 0.5032619047619046
Linear Regression MSE: 1.0614977440363134
Decision Tree Regression MSE: 0.6666666666666666
Gradient Boosting Regression MSE: 0.6289341958263573
K-Nearest Neighbors Regression MSE: 0.9542857142857144


e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [12]:
models = {
    'Random Forest Classifier': RandomForestClassifier(random_state=42),
    'Support Vector Classifier': SVC(kernel='linear', random_state=42),
    'K-Nearest Neighbors Classifier': KNeighborsClassifier()
}
classification_reports = {}

In [16]:
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    classification_reports[model_name] = report
    print(f"{model_name} accuracy: {accuracy}")
    print(report)

e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: 

Random Forest Classifier accuracy: 0.7619047619047619
              precision    recall  f1-score   support

           0       0.86      0.92      0.89        13
           1       0.50      1.00      0.67         2
           2       1.00      0.33      0.50         3
           3       0.50      1.00      0.67         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1

    accuracy                           0.76        21
   macro avg       0.48      0.54      0.45        21
weighted avg       0.74      0.76      0.72        21

Support Vector Classifier accuracy: 0.8095238095238095
              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.40      1.00      0.57         2
           2       1.00      0.33      0.50         3
           3       0.50      1.00      0.67         1
           4       1.00      1.00      1.00         1
           5       1.00

e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Pre

In [15]:
classification_reports

{}

In [14]:
print(classification_reports['Support Vector Classifier'])
print(classification_reports['Random Forest Classifier'])
print(classification_reports['K-Nearest Neighbors Classifier'])

KeyError: 'Support Vector Classifier'

In [41]:
joblib.dump(models['Support Vector Classifier'], 'svm_model.joblib')

['svm_model.joblib']

In [42]:
joblib.dump(encoder, 'encoder.joblib')


['encoder.joblib']

In [51]:
data = X.head(1)

In [47]:
model1 = joblib.load('svm_model.joblib')
encoder1 = joblib.load('encoder.joblib')

In [52]:
encoded_data = encoder1.transform(data[categorical_cols])
encoded_cols = encoder1.get_feature_names_out(categorical_cols)
encoded_data_df = pd.DataFrame(encoded_data.toarray(), columns=encoded_cols)

data_numerical = data.drop(columns=categorical_cols)
data_final = pd.concat([encoded_data_df, data_numerical], axis=1)

print(model1.predict(data_final))

[5]
